In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch.serialization import add_safe_globals

# Custom dataset class to hold preprocessed image data
class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform

    def __getitem__(self, index):
        img = self.imgs[index]
        label = self.labels[index]

        # Convert non-RGB images to RGB
        if img.mode != "RGB":
            img = img.convert("RGB")

        # Apply transformation if provided
        if self.transform:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return len(self.imgs)

# Register custom class for loading serialized dataset
add_safe_globals({'TaskDataset': TaskDataset})

# Load the serialized dataset
dataset = torch.load("/kaggle/input/traindata/Train.pt", weights_only=False)

# Define image transformations (resize and normalize)
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])
dataset.transform = transform

# Create data loader for batching
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# FGSM attack implementation
def fgsm_attack(model, images, labels, epsilon):
    images.requires_grad = True
    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)
    model.zero_grad()
    loss.backward()
    # Add adversarial noise
    return torch.clamp(images + epsilon * images.grad.sign(), 0, 1)

# PGD attack implementation
def pgd_attack(model, images, labels, epsilon=0.03, alpha=0.01, steps=3):
    ori = images.clone().detach()
    images = ori + 0.001 * torch.randn_like(ori)  # Random start
    for _ in range(steps):
        images.requires_grad = True
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        # Perform iterative update
        images = images + alpha * images.grad.sign()
        eta = torch.clamp(images - ori, min=-epsilon, max=epsilon)
        images = torch.clamp(ori + eta, min=0, max=1).detach()
    return images

# Load and modify ResNet18 for 10-class classification
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 10)

# Move model to available device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training configuration
epochs = 15
epsilon = 0.03   # FGSM/PGD max perturbation
alpha = 0.01     # PGD step size
pgd_steps = 3    # PGD steps

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Clean loss
        out_clean = model(images)
        loss_clean = criterion(out_clean, labels)

        # FGSM loss
        images_fgsm = fgsm_attack(model, images, labels, epsilon)
        out_fgsm = model(images_fgsm)
        loss_fgsm = criterion(out_fgsm, labels)

        # PGD loss
        images_pgd = pgd_attack(model, images, labels, epsilon, alpha, pgd_steps)
        out_pgd = model(images_pgd)
        loss_pgd = criterion(out_pgd, labels)

        # Combine and backpropagate
        loss = (loss_clean + loss_fgsm + loss_pgd) / 3
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1:3d}/{epochs} - Avg Loss: {avg_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "robust_model.pt")
print("Model saved as 'robust_model.pt'")


In [ ]:
#### Tests ####
# (these are the assertions being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("/kaggle/working/robust_model.pt", "rb") as f:
    try:
        model: torch.nn.Module = allowed_models["resnet18"](weights=None)
        model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    except Exception as e:
        raise Exception(
            f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
        )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")



In [ ]:
# --- Submission ---
import requests
response = requests.post(
    "http://34.122.51.94:9090/robustness",
    files={"file": open("/kaggle/working/robust_model.pt", "rb")},
    headers={"token": "12910150", "model-name": "resnet18"}
)
print("Submission response:", response.json())